Casava 1.8 FASTQ ID format from 
https://en.wikipedia.org/wiki/FASTQ_format

@EAS139:136:FC706VJ:2:2104:15343:197393 1:Y:18:ATCACG
                                    
* EAS139	the unique instrument name
* 136	the run id
* FC706VJ	the flowcell id
* 2	flowcell lane
* 2104	tile number within the flowcell lane
* 15343	'x'-coordinate of the cluster within the tile
* 197393	'y'-coordinate of the cluster within the tile
* 1	the member of a pair, 1 or 2 (paired-end or mate-pair reads only)
* Y	Y if the read is filtered, N otherwise
* 18	0 when none of the control bits are on, otherwise it is an even number
* ATCACG	index sequence

The sequence of quality values (from left to right):
```
!"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~
```


In [11]:
import random

nucleotides = ["A", "C", "T", "G"]

def random_nucleotide_sequence(length):
    return "".join(random.choices(nucleotides, k=length))

print(random_nucleotide_sequence(10))
print(random_nucleotide_sequence(1))
print(random_nucleotide_sequence(100))

                

AGTAGTCGAT
G
CTGCGCGACCGTATCCTTTCGCGGGGCTCATGGGGCAAGGTACGAACAGTAGCATGAAACACCGGCACAGCTATAGACTGCTAGATCCACTCGATATCAT


In [39]:
quality_chars = "!\"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~"

def random_qualities(
        length,
        min_quality=0,
        max_quality=len(quality_chars)):
    return "".join(random.choices(quality_chars[min_quality:max_quality], k=length))

print(random_qualities(10))
print(random_qualities(1))
print(random_qualities(100))


print(random_qualities(10, min_quality=20))
print(random_qualities(1, min_quality=20))

print(random_qualities(100, min_quality=20))

RbnCspL80E
z
Ax0cz#og>4dX@,`Y<9(`_Ag5fVg~M)!?J%:17,]Vk/,90Bf7yt%>D9YLq[c;!*NvI&@CoERkvZ|#+a&BF#@N=UATAJtyU@O(//|T
RR?|aUt?~6
x
pa6C=5?MLnp=mnW9r9OLa=DbT<bjszavuGzvsULL?:[DxkSqWNF[Kar;8aLfNuTNJwvgMvhwDcJ:HC>TDmM>=?5LCN|_>W6`B5~A


In [86]:
class ReadGenerator(object):
    def __init__(
            self,
            instrument_name="HISEQ1",
            run_id=1,
            flowcell_id="FC706VJ",
            lane=1,
            max_tile_number=9999,
            max_x_coord=2 * 10 ** 4,
            max_y_coord=2 * 10 ** 5,
            pair_number=1,
            filtered_reads=False,
            control_bits=18,
            index_length=6,
            min_base_quality=20,
            max_base_quality=40):
        self.instrument_name = instrument_name
        self.run_id = run_id
        self.flowcell_id = flowcell_id
        self.lane = lane
        self.max_tile_number = max_tile_number
        self.max_x_coord = max_x_coord
        self.max_y_coord = max_y_coord
        self.pair_number = pair_number
        self.filtered_reads = filtered_reads
        self.control_bits = control_bits
        self.index_length = index_length
        self.min_base_quality = min_base_quality
        self.max_base_quality = max_base_quality
        
    def _random_index(self):
        return random_nucleotide_sequence(self.index_length)
    
    def generate_reads(self, sequence, count, read_length=50):
        """
        Returns list of tuples whose elements are
            1) id string
            2) read sequence
            3 ) quality scores
        """
        assert len(sequence) >= read_length
        fastq_entries = []
        index = self._random_index()
        for i in range(count):
            offset = random.randint(0, len(sequence) - read_length)
            read_seq = sequence[offset:offset + read_length]
            qualities = random_qualities(
                length=read_length, 
                min_quality=self.min_base_quality,
                max_quality=self.max_base_quality)
            tile_number = random.randint(1, self.max_tile_number)
            x_coord = random.randint(1, self.max_x_coord)
            y_coord = random.randint(1, self.max_y_coord)
            id_string = "@%s:%d:%s:%d:%d:%d:%d %d:%s:%d:%s length=%d" % (
                self.instrument_name,
                self.run_id,
                self.flowcell_id,
                self.lane, 
                tile_number,
                x_coord,
                y_coord,
                self.pair_number,
                "Y" if self.filtered_reads else "N",
                self.control_bits,
                index,
                read_length)
            fastq_entry = (id_string, read_seq, qualities)
            fastq_entries.append(fastq_entry)
        return fastq_entries
    
    def generate_fastq_string(self, sequence, count, read_length=50):
        fastq_entries = self.generate_reads(sequence=sequence, count=count, read_length=read_length)
        return "\n".join([
            "%s\n%s\n+\n%s" % (id_string, seq, qual) for (id_string, seq, qual) in fastq_entries])
    
    def generate_fastq_strings_for_variant(self, variant, ref_count, alt_count, read_length=50):
        effect = variant.effects().top_priority_effect()
        transcript = effect.transcript
        transcript_sequence = transcript.sequence
        variant_start_offset = transcript.spliced_offset(variant.start)
        variant_end_offset = transcript.spliced_offset(variant.end)
        
        prefix = transcript_sequence[
            max(0, variant_start_offset - read_length):variant_start_offset - 1]
        
        suffix = transcript_sequence[
            variant_end_offset + 1:variant_end_offset + read_length]
        
        ref_sequence = prefix + variant.ref + suffix
        alt_sequence = prefix + variant.alt + suffix
        
        ref_fastq_string = self.generate_fastq_string(
            sequence=ref_sequence,
            count=ref_count,
            read_length=read_length)
        alt_fastq_string = self.generate_fastq_string(
            sequence=alt_sequence,
            count=alt_count,
            read_length=read_length)
        return ref_fastq_string, alt_fastq_string


In [87]:
r = ReadGenerator()

In [88]:
print(r.generate_fastq_string("ACTGAACCTTGGAAACCCTTTGGG", count=2, read_length=5))

@HISEQ1:1:FC706VJ:1:9866:1635:68955 1:N:18:TGTTAC length=5
CTTGG
+
@;9A7
@HISEQ1:1:FC706VJ:1:2775:5972:54953 1:N:18:TGTTAC length=5
TGGAA
+
H6C>A


In [89]:
from varcode import Variant

In [83]:
idh1_r132h = Variant(2, 209113112, "C", "T", ensembl="grch37")

In [90]:
idh1_r132h.effects().top_priority_effect()

Substitution(variant=Variant(contig='2', start=209113112, ref='C', alt='T', reference_name='GRCh37'), transcript_name=IDH1-006, transcript_id=ENST00000415913, effect_description=p.R132H)

In [91]:
ref_fastq, alt_fastq = r.generate_fastq_strings_for_variant(variant=idh1_r132h, ref_count=50, alt_count=100)

In [94]:
print(ref_fastq)

@HISEQ1:1:FC706VJ:1:3648:15682:76608 1:N:18:TGAGAA length=50
GTGGATGGGTAAAACCTATCATCATAGGTCTCATGCTTATGGGGATCAAT
+
G=B6<=@:H:66FF?<==HE6=C:8?B6HA6>B<A@GDCB8775B9<:A6
@HISEQ1:1:FC706VJ:1:3795:5545:131311 1:N:18:TGAGAA length=50
TCATCATAGGTCTCATGCTTATGGGGATCAATACAGAGCAACTGATTTTG
+
=GF7D9E8>;5@6>@>FE7G>CC5DHG7DB5CFD9856A?B:9FAHG9F=
@HISEQ1:1:FC706VJ:1:4121:5300:84217 1:N:18:TGAGAA length=50
CATCATAGGTCTCATGCTTATGGGGATCAATACAGAGCAACTGATTTTGT
+
;;E:6D=:C6?7?G6;?H<@>88=<8>;?CD:H>>:DCD;G>G>;78A9<
@HISEQ1:1:FC706VJ:1:9887:5035:156345 1:N:18:TGAGAA length=50
CGGCTTGTGAGTGGATGGGTAAAACCTATCATCATAGGTCTCATGCTTAT
+
7A:@A7D:;;8E9CCCA??G9=7>BEH>;859@::>9>8D5BFB5B?5;<
@HISEQ1:1:FC706VJ:1:1577:10840:155148 1:N:18:TGAGAA length=50
GAGTGGATGGGTAAAACCTATCATCATAGGTCTCATGCTTATGGGGATCA
+
C<AA:<C:C75C@G<@5;88HF@7B6E788<5@D8>H:9=;D<@HFDF<E
@HISEQ1:1:FC706VJ:1:6374:11329:137806 1:N:18:TGAGAA length=50
TAGGTCTCATGCTTATGGGGATCAATACAGAGCAACTGATTTTGTTGTTC
+
G@;7EE5F@?BD:6>?57:H=A7C=E8>D5<9EH;9A8HB>>5>57>@<C
@HISEQ1:1

In [95]:
print(alt_fastq)

@HISEQ1:1:FC706VJ:1:8608:15315:4350 1:N:18:GCTCGC length=50
GGTAAAACCTATCATCATAGGTTTCATGCTTATGGGGATCAATACAGAGC
+
G5E5?G:F=HDD5E@A;<B5FABA;9B8D>;F85>?>;H>BDCG;@9:77
@HISEQ1:1:FC706VJ:1:3568:16868:136747 1:N:18:GCTCGC length=50
CCCCGGCTTGTGAGTGGATGGGTAAAACCTATCATCATAGGTTTCATGCT
+
EH9A5=D5=GD:;58HE=>=:787E@=CGGG=;A;D:><?5B86FHHG==
@HISEQ1:1:FC706VJ:1:6344:15558:94156 1:N:18:GCTCGC length=50
AACCTATCATCATAGGTTTCATGCTTATGGGGATCAATACAGAGCAACTG
+
F?<68F@A<>;@5FAE;9GAE<D6@@FE>E7F5879?:5H:D;B77;ACC
@HISEQ1:1:FC706VJ:1:79:18978:118895 1:N:18:GCTCGC length=50
GCTTGTGAGTGGATGGGTAAAACCTATCATCATAGGTTTCATGCTTATGG
+
=8F9;<G@:A96><?E:=6;:5:E8G@8F?7=:C<=:BC=D;B7>G67HB
@HISEQ1:1:FC706VJ:1:264:6432:6448 1:N:18:GCTCGC length=50
ATCATAGGTTTCATGCTTATGGGGATCAATACAGAGCAACTGATTTTGTT
+
CH7:D@8EF<?79:B?G>HBDC<EHDA56C7@58A<<E;EF@F?=7H968
@HISEQ1:1:FC706VJ:1:3648:17795:52556 1:N:18:GCTCGC length=50
TCATCATAGGTTTCATGCTTATGGGGATCAATACAGAGCAACTGATTTTG
+
F;=9;;8FH57?>;7GG>G5A<FHB:97<5CAGDH=765D>>?6<DA>9>
@HISEQ1:1:FC70

In [96]:
with open("normal.fastq", "w") as f:
    f.write(ref_fastq)
with open("tumor.fastq", "w") as f:
    f.write(alt_fastq)